In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(False)

In [2]:
client = MongoClient()
loladb = client.loladb73
summoners_collection = loladb.summoners
matches_collection = loladb.matches
print summoners_collection.count()
print matches_collection.count()

2829
30993


In [3]:
champions = riotapi.get_champions()
dict_id2idx = dict()
dict_idx2id = dict()
dict_id2key = dict()
dict_key2idx = dict()
dict_idx2key = dict()
for a_idx, a_champion in enumerate(champions):
    champ_id = a_champion.id
    champ_key = a_champion.key
    dict_id2idx[champ_id] = a_idx
    dict_idx2id[a_idx] = champ_id
    dict_id2key[champ_id] = champ_key
    dict_key2idx[champ_key.lower()] = a_idx
    dict_idx2key[a_idx] = champ_key.lower()
    
#print dict_id2key[dict_idx2id[132]]
#print dict_id2key

def GetIdxFromId(a_id):
    return dict_id2idx[a_id]

def GetIdFromIdx(a_idx):
    return dict_idx2id[a_idx]

def GetNameFromId(a_id):
    return dict_id2key[a_id]

def TranslateName(a_key):
    if a_key == "wukong":
        return "monkeyking"
    return a_key

print dict_key2idx

{u'evelynn': 125, u'zed': 127, u'nunu': 95, u'swain': 63, u'taric': 130, u'skarner': 91, u'ekko': 100, u'nasus': 67, u'graves': 27, u'nocturne': 77, u'shaco': 2, u'morgana': 72, u'aatrox': 104, u'draven': 105, u'nidalee': 4, u'kassadin': 37, u'sona': 38, u'monkeyking': 0, u'reksai': 116, u'cassiopeia': 42, u'taliyah': 81, u'malzahar': 112, u'amumu': 84, u'khazix': 47, u'olaf': 48, u'sivir': 17, u'riven': 18, u'nami': 119, u'jhin': 120, u'kogmaw': 102, u'gangplank': 20, u'syndra': 50, u'poppy': 22, u'volibear': 55, u'kennen': 57, u'ryze': 59, u'gragas': 126, u'garen': 94, u'shen': 60, u'karthus': 23, u'jayce': 24, u'bard': 64, u'sion': 65, u'elise': 97, u'alistar': 98, u'leblanc': 132, u'tahmkench': 106, u'ezreal': 133, u'fiddlesticks': 68, u'lux': 30, u'fiora': 33, u'kalista': 35, u'twistedfate': 69, u'kled': 6, u'illaoi': 9, u'kindred': 41, u'azir': 79, u'drmundo': 51, u'tryndamere': 13, u'hecarim': 46, u'lissandra': 113, u'darius': 12, u'yorick': 15, u'jax': 1, u'jarvaniv': 118, u'as

In [4]:
summoner_dict = {}
for a_idx, a_match in enumerate(matches_collection.find()):
    participants = a_match['participantIdentities']
    for a_participant in participants:
        p_id = a_participant['player']['summonerId']
        p_name = a_participant['player']['summonerName']
        if p_id not in summoner_dict.keys():
            summoner_dict[p_id] = p_name
    if a_idx % 1000 == 0:
        print a_idx

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000


In [8]:
print len(summoner_dict)
all_summoners_collection = loladb.all_summoners
all_summoners_collection.insert_many([{str(each_id):summoner_dict[each_id]} for each_id in summoner_dict])

44101


In [11]:
summoner_size = len(summoner_dict)
feature_size = 134
X = np.zeros((summoner_size, feature_size))
summoners_history_collection = loladb.summoners_history

for a_idx, a_summoner_id in enumerate(summoner_dict):   
    a_summoner_name = summoner_dict[a_summoner_id]
    #print a_summoner_id, a_summoner_name
    champion_url = "https://www.op.gg/summoner/champions/userName="+a_summoner_name
    r = requests.get(champion_url)
    soup = BeautifulSoup(r.text)
    season7_part = soup.find("table", class_="ChampionStatsTable")
    if season7_part == None:
        continue
    season7_body = season7_part.find("tbody", class_="Body")
    champion_items = season7_body.find_all("tr", class_="Row")
    champions_dict = {}
    for champion_item in champion_items:
        champion_name_part = champion_item.find("td", class_="ChampionName")
        champion_name = champion_name_part.find('a').get_text()
        neat_name = TranslateName(champion_name.lower().replace(" ", "").replace("'", "").replace(".", ""))
        champion_rate_part = champion_item.find("div", class_="Graph")
        champion_win = 0
        champion_lose = 0
        text_left = champion_rate_part.find("div", class_="Text Left")
        if text_left != None:
            champion_win_text = text_left.get_text()[0:-1]
            champion_win = int(champion_win_text)
        text_right = champion_rate_part.find("div", class_="Text Right")
        if text_right != None:
            champion_lose_text = text_right.get_text()[0:-1]
            champion_lose = int(champion_lose_text)
        #print champion_name, champion_win, champion_lose
        champions_dict[neat_name] = champion_win + champion_lose
    summoners_history_collection.insert(champions_dict)
    if False:
        total_num = 0
        for a_champion in champions_dict:
            total_num += champions_dict[a_champion]
        for a_champion in champions_dict:
            a_neat_champion = a_champion.lower().replace(" ", "").replace("'", "").replace(".", "")
            #print a_neat_champion
            champion_idx = dict_key2idx[TranslateName(a_neat_champion)]
            champion_ratio = champions_dict[a_champion] / float(total_num)
            X[a_idx, champion_idx] = champion_ratio
    if a_idx % 10 == 0:
        print a_idx, a_summoner_id, a_summoner_name

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


0 55050242 KDHEART
10 5636124 북방 졍찰대 퀸
20 6160438 엘로힘
30 2097254 Xin jiang9
40 8913028 LET ME JINX
50 6946993 Maestro AD
60 1966290 김유한
70 12321011 급 유
80 3784048 마더핸드파이
90 55181608 긴 건 붕
100 30802250 문동현5
110 41681257 봇가면미드로던짐
120 16860155 CARETT
130 15860135 블리치크랭크
140 19530182 고  치
150 3801558 살 려 줘 꾸엑
160 5112313 군대가기일보직전
170 17695242 DJ 꽃
180 10617398 RagingRag1nG
190 11141716 재밋게게임하자요
200 3801701 핑으로말해줘
210 13534994 까까먹는애니
220 11403937 입털면겜안해요
230 42074812 TOP Yassuo
240 7733965 팥빠진단팥죽
250 8389369 모 캔
260 2491177 정글수준오짐
270 24511310 몌 룡
280 2753378 디나흐트
290 8651663 똥케리
300 8651690 윤 쩡글
310 4588484 문휘성
320 9176031 에이프런
330 32113661 이동룡
340 2491416 zzzzzzzzzzzzz2
350 7734332 Tong   Jia  Wei
360 48890973 DeMarco
370 3933295 탑승중
380 8651938 G5o Back
390 5811060 only sp blitz
400 2381348 켄치 왕
410 3802360 CHAOS인
420 14026000 지인낭자
430 2884906 넨 이
440 17171800 Victory Mode
450 44041587 아니 그걸 왜 후
460 11273619 NANaANT
470 13370820 파소엘
480 31852006 Ria triumph
490 1312270 WUZU
500 3409444 V

ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [21]:
sample_size = summoners_history_collection.count()
feature_size = 134
X = np.zeros((sample_size, feature_size))
for a_idx, a_history in enumerate(summoners_history_collection.find()):
    total_num = 0
    for a_champion in a_history:
        if a_champion == u'_id':
            continue
        total_num += a_history[a_champion]
    for a_champion in a_history:
        if a_champion == u'_id':
            continue
        champion_idx = dict_key2idx[a_champion]
        champion_ratio = a_history[a_champion] / float(total_num)
        X[a_idx, champion_idx] = champion_ratio
        
    if a_idx % 1000 == 0:
        print a_idx

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [29]:
from sklearn.cluster import KMeans
A = np.zeros((5, 2))
A[0, :] = [2, 2]
A[1, :] = [2.5, 2]
A[2, :] = [2, 2.2]
A[3, :] = [5, 4]
A[4, :] = [4, 6]
km = KMeans(n_clusters = 2)
km.fit(A)
print km.cluster_centers_

[[ 4.5         5.        ]
 [ 2.16666667  2.06666667]]


In [84]:
lolkm = KMeans(n_clusters=5)
lolkm.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [73]:
#print lolkm.cluster_centers_
cluster_index = 4
for champ_index in range(feature_size):
    champ_name = dict_idx2key[champ_index]
    cluster_center = lolkm.cluster_centers_[cluster_index]
    if cluster_center[champ_index] > 0.02:
        print champ_name, cluster_center[champ_index]

zyra 0.06139647409
missfortune 0.0418744302053
blitzcrank 0.0801904193499
thresh 0.0498599919646
karma 0.0851825732538
morgana 0.0277405928174
leona 0.0380635589489
alistar 0.0294018894229
malzahar 0.0352409765464
nami 0.0592692503399
soraka 0.0517765007965
janna 0.0457277115909
lulu 0.0350865183092


In [85]:
print lolkm.

<bound method KMeans.score of KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)>


In [67]:
for i in range(5):
    selected = (lolkm.labels_ == i)
    print sum(selected)

999
4415
15420
5504
3168


In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt

range_n_clusters = [2, 3]

for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(A)
    
    silhouette_avg = silhouette_score(A, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)
    
    sample_silhouette_values = silhouette_samples(A, cluster_labels)
    
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    plt.show()
    break

('For n_clusters =', 2, 'The average silhouette_score is :', 0.70283828411209637)
